<a href="https://colab.research.google.com/github/AI-Isaiah/AI-Isaiah/blob/main/20230329_9_ensemble_method_with_smote_optuna%2Bcost_function_AFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
!pip install optuna
!pip install catboost
#!pip install verstack
#from verstack import LGBMTuner
import warnings
import seaborn as sns
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping, log_evaluation
from sklearn.pipeline import Pipeline
import sys
import lightgbm as lgbm
import logging
sys.path.append('/bin/sample_data')
sys.path.append('/root')
#from preprocessor import generate_cm
from collections import Counter
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
import optuna
import pickle
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, RandomizedSearchCV, ParameterSampler, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, precision_score, log_loss, f1_score, roc_auc_score, roc_curve, recall_score, accuracy_score, make_scorer,ConfusionMatrixDisplay,matthews_corrcoef, classification_report
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import gc
import json
from imblearn.over_sampling import SMOTE, ADASYN, SMOTENC


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.9 MB/s eta 0:00:00


In [3]:
!nvidia-smi

Fri Apr  7 15:15:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
def generate_cm1(y_true, y_pred, title):
    y_pred = (y_pred > 0.5).astype(int)
    cm = confusion_matrix(y_true, y_pred)

    # Display confusion matrix using ConfusionMatrixDisplay
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Class 0", "Class 1"])
    disp.plot()
    counter = Counter(y_true)
    for key, value in counter.items():
      print(f'{title}"{key}": {value}')


In [9]:
data = pd.read_csv("/content/sample_data/loan_book_data_labels.csv")
labels = data["Result"]
data.drop("Result", axis=1, inplace=True)
print(labels.head())
print(data.head())

0    1
1    1
2    1
3    1
4    1
Name: Result, dtype: int64
  v01_src        v02_src  v03_src               v04_src v05_src v06_src  \
0  new_id       Personal       30  University education    Male  Father   
1  new_id       Personal       30  University education    Male  Father   
2  old_id  Bills payment       25       Basic education    Male  Mother   
3  old_id       Personal       35       Basic education    Male  Father   
4  old_id       Personal       30  University education  Female  Father   

   v07_src  v08_src   v09_src       v10_src  ...  v60_src v61_src v62_src  \
0  Married        2  18000000  SelfEmployed  ...       -1      -1      -1   
1  Married        1  12000000      Salaried  ...       -1       2       4   
2   Single        0  10000000  SelfEmployed  ...       -1      -1      -1   
3   Single        0  12000000      Salaried  ...       -1      -1      -1   
4  Married        0  12000000      Salaried  ...        1       2       2   

   v63_src  v64_src  v65

In [ ]:
def objective_xgboost1(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'gamma': trial.suggest_float('gamma', 1e-9, 100),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 100),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 100),
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "random_state": 42,
    }
            
    
    # Create an XGBClassifier with the specified hyperparameters
    clf = xgb.XGBClassifier(**params, tree_method="gpu_hist", enable_categorical=True, max_cat_to_onehot=1)
    clf.fit(X, y, verbose=False)
    y_pred = clf.predict(X)
    y_pred = (y_pred > 0.5).astype(int)
    return accuracy_score(y, y_pred)

def objective_low_level(trial, X, y):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'gamma': trial.suggest_float('gamma', 1e-9, 100),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 100),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 100),
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "random_state": 42,
    }
 
    # Create an XGBClassifier with the specified hyperparameters
    #clf = xgb.XGBClassifier(**params,enable_categorical=True)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    #X = X.values   
    
    dtrain = xgb.DMatrix(X, y, enable_categorical=True)
    
    bst = xgb.train(params, dtrain, 500, verbose_eval=False, tree_method="gpu_hist", max_cat_to_onehot=1)
    #scores = cross_val_score(bst, X, y, cv=skf, scoring='accuracy')

    y_pred = bst.predict(dtrain) #ntree_limit=bst.best_ntree_limit)
        
    y_pred = (y_pred > 0.5).astype(int)
    scores.append(accuracy_score(y, y_pred))
    #params['n_estimators'] = num_round


    
    return np.mean(scores)

   

def feature_selection2(X, y, max_features=30, n_trials=50):
    # Split data for hyperparameter tuning
    #X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
     # Create a DMatrix for training
    """dtrain = xgb.DMatrix(data=X_train, label=y_train, enable_categorical=True)
    dtest = xgb.DMatrix(data=X_valid, label=y_valid, enable_categorical=True)
    """
    # Optimize hyperparameters using Optuna
    
    study = optuna.create_study(direction='maximize', sampler = optuna.samplers.NSGAIISampler(seed = 42)) # update2
    study.optimize(lambda trial: objective_xgboost1(trial, X,y), n_trials=n_trials)
    best_params = study.best_params
    print('Best hyperparameters: {}'.format(best_params))

    # Train initial XGBoost model with the best hyperparameters
    #num_round = best_params.pop('n_estimators')  # Remove n_estimators from params and store its value
    #init_clf = xgb.train(best_params, dtrain, num_round, [(dtest, 'validation')], early_stopping_rounds=100, verbose_eval=False)
    init_clf = xgb.XGBClassifier(**best_params, tree_method="gpu_hist", enable_categorical=True, max_cat_to_onehot=1)
    init_clf.fit(X,y)

    # Get feature importances and indices
    """importances = init_clf.get_score(importance_type='weight')
    indices = sorted(importances, key=lambda x: importances[x], reverse=True)
    """
    importances = init_clf.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Select the top-k features
    selected_features = indices[:max_features]
    print("Indices in function: ", indices)
    #X_selected = X[:, selected_features]
    

    return selected_features



In [14]:
X = data
if "appl_ref" in data.columns:
    # drop the column
    data.drop("appl_ref", axis=1, inplace=True)

for i in range(52, 70):
  column_name = "v" + str(i) + "_src"
  if column_name in data.columns:
    data.drop(column_name, axis=1, inplace=True)

X = data
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
print(X.dtypes)
feature_names = X.columns
le = LabelEncoder()
y = le.fit_transform(labels)
y = y.ravel()
cat_columns = X.select_dtypes(include='category').columns

num_columns = X.select_dtypes(exclude=['category']).columns

# Encode categorical columns using LabelEncoder
"""label_encoders = {}
for col in cat_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le"""



IndentationError: ignored

In [6]:
class NumericalScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        self.num_cols = []

    def fit(self, X, y=None):
        self.num_cols = X.select_dtypes(include=['int64', 'float64']).columns
        self.scaler.fit(X[self.num_cols])
        return self

    def transform(self, X):
        X_scaled = X.copy()
        X_scaled[self.num_cols] = self.scaler.transform(X[self.num_cols])
        return X_scaled

In [9]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

X_train = pd.DataFrame(X_train, columns=X.columns)
y_train = pd.DataFrame(y_train, columns=["Result"])
X_val = pd.DataFrame(X_val, columns=X.columns)
y_val = pd.DataFrame(y_val, columns=["Result"])
X_test = pd.DataFrame(X_test, columns=X.columns)
y_test = pd.DataFrame(y_test, columns=["Result"])

#selected_features = feature_selection2(X_train, y_train, 30, 50)

#X_train = X_train.iloc[:, selected_features]
#X_val = X_val.iloc[:, selected_features]
#X_test = X_test.iloc[:, selected_features]

# Identify categorical columns
cat_cols = X_train.select_dtypes(include=['category']).columns.tolist()
print("Cat-cols: ", cat_cols)
num_cols = X_train.select_dtypes(exclude=['category']).columns.tolist()
# Create a SMOTENC object with categorical features
smote_nc = SMOTENC(categorical_features=[X_train.columns.get_loc(c) for c in cat_cols], random_state=42)

# Fit the SMOTENC object to the training data
X_train, y_train = smote_nc.fit_resample(X_train, y_train)

print("X_train_shape final: ", X_train.shape)
print("X_train dtypes:" , X_train.dtypes)


Cat-cols:  ['v02_src', 'v04_src', 'v05_src', 'v06_src', 'v07_src', 'v10_src', 'v12_src', 'v13_src', 'v18_src', 'v19_src', 'v20_src', 'v21_src', 'v22_src', 'v41_src', 'v42_src', 'v43_src']
X_train_shape final:  (1414, 68)
X_train dtypes: v02_src    category
v03_src       int64
v04_src    category
v05_src    category
v06_src    category
             ...   
v65_src       int64
v66_src     float64
v67_src     float64
v68_src       int64
v69_src       int64
Length: 68, dtype: object


In [10]:
preprocessor = NumericalScaler()

# Fit the preprocessor on the training data
X_train= preprocessor.fit_transform(X_train)

# Apply the preprocessor on the validation and test data
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

print("X_train - dtypes: ", X_train.dtypes)
print("X_vak - dtypes: ", X_val.dtypes)


X_train - dtypes:  v02_src    category
v03_src     float64
v04_src    category
v05_src    category
v06_src    category
             ...   
v65_src     float64
v66_src     float64
v67_src     float64
v68_src     float64
v69_src     float64
Length: 68, dtype: object
X_vak - dtypes:  v02_src    category
v03_src     float64
v04_src    category
v05_src    category
v06_src    category
             ...   
v65_src     float64
v66_src     float64
v67_src     float64
v68_src     float64
v69_src     float64
Length: 68, dtype: object


In [ ]:

def custom_loss(y_pred, y_true):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return (tn * 4) - (fn * 10)

# Define the objective function for Optuna
def objective_catboost(trial, X, y, cat_cols):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'depth': trial.suggest_int('depth', 2, 10),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 5, 200),
        'random_strength': trial.suggest_int('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.00),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 1, 10),
        'task_type': 'GPU'
    }
    
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Train and evaluate the model using cross-validation
    scores = []
    for train_idx, val_idx in skf.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        # Create a Pool object with categorical features
        pool_train = Pool(X_train, y_train, cat_features=cat_cols)
        pool_valid = Pool(X_val, y_val, cat_features=cat_cols)
        
        # Train the model
        model = CatBoostClassifier(**params, random_state = 42)
        model.fit(pool_train, eval_set=pool_valid, verbose=False)
        
        # Evaluate the model on the validation set
        y_pred = model.predict(X_val)
        y_pred = (y_pred > 0.5).astype(int)
        score = custom_loss(y_val, y_pred)
        scores.append(score)

    return np.mean(scores)

#Starting hyperparamters to improve convergence
starting_cat_hp = {'iterations': 109, 
                            'learning_rate': 0.45363716844543917, 
                            'depth': 2, 
                            'l2_leaf_reg': 4, 
                            'border_count': 191, 
                            'random_strength': 20, 
                            'bagging_temperature': 57.34805443344737, 
                            'grow_policy': 'SymmetricTree', 
                            'max_ctr_complexity': 4
}

# Optimize hyperparameters

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.NSGAIISampler(seed = 42))
study.enqueue_trial(starting_cat_hp)
study.optimize(lambda trial: objective_catboost(trial, X_train,y_train, cat_cols), n_trials=100)





[I 2023-04-05 08:50:40,667] A new study created in memory with name: no-name-5ff5bcac-63b2-4a41-8e42-7e482732edea
[I 2023-04-05 08:51:35,004] Trial 0 finished with value: 324.8 and parameters: {'iterations': 437, 'learning_rate': 0.4758500101408589, 'depth': 8, 'l2_leaf_reg': 6, 'border_count': 35, 'random_strength': 4, 'bagging_temperature': 5.817780380698264, 'grow_policy': 'SymmetricTree', 'max_ctr_complexity': 1}. Best is trial 0 with value: 324.8.
[I 2023-04-05 08:52:59,077] Trial 1 finished with value: -22.4 and parameters: {'iterations': 973, 'learning_rate': 0.41789689399220664, 'depth': 3, 'l2_leaf_reg': 2, 'border_count': 40, 'random_strength': 7, 'bagging_temperature': 52.48039559890746, 'grow_policy': 'Lossguide', 'max_ctr_complexity': 2}. Best is trial 0 with value: 324.8.
[I 2023-04-05 08:53:27,239] Trial 2 finished with value: -229.6 and parameters: {'iterations': 363, 'learning_rate': 0.18951730321390894, 'depth': 6, 'l2_leaf_reg': 8, 'border_count': 44, 'random_strengt

In [ ]:
# Get best parameters
best_params = study.best_params
print("Best parameters: ", best_params)

#Pool variables to be used with catboost:
pool_train = Pool(X_train, y_train, cat_features = cat_cols)
pool_valid = Pool(X_val, cat_features = cat_cols)
pool_test = Pool(X_test, cat_features = cat_cols)


# Run the randomized search
# best parameters: iterations': 420, 'learning_rate': 0.18951730321390894, 'depth': 2, 'l2_leaf_reg': 2, 'border_count': 44, 'random_strength': 11, 'bagging_temperature': 59.24553274051562, 'grow_policy': 'SymmetricTree', 'max_ctr_complexity': 5
cb_classifier = CatBoostClassifier(**best_params, random_state=42, verbose=0)
cb_classifier.fit(pool_train)
tn, fp, fn, tp = confusion_matrix(y_val, cb_classifier.predict(X_val)).ravel()
print("confusion Matrix validation")
print("tn: ", tn, "fp: ", fp, "fn: ", fn, "tp: ", tp)
generate_cm(y_val, cb_classifier.predict(pool_valid), "CatBoost - Validation")
tn, fp, fn, tp = confusion_matrix(y_test, cb_classifier.predict(X_test)).ravel()
print("confusion Matrix test")
print("tn: ", tn, "fp: ", fp, "fn: ", fn, "tp: ", tp)
generate_cm(y_test, cb_classifier.predict(pool_test), "CatBoost - Test")

# Save the best parameters for CatBoost
with open('cat_best_params.json', 'w') as f:
    json.dump(study.best_params, f)

# Clear memory
del cb_classifier
gc.collect()

NameError: ignored

In [ ]:
# Create a RandomForest classifier
rf_clf = RandomForestClassifier(random_state=42)

# Define the parameter space for RandomForest
rf_param_grid = {
    'n_estimators': list(range(50, 500)),
    'max_depth': list(range(1, 50)),
    'min_samples_split': list(range(2, 50)),
    'min_samples_leaf': list(range(1, 50)),
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
}

# Set up randomized search for RandomForest
rf_random_search = RandomizedSearchCV(
    estimator=rf_clf,
    param_distributions=rf_param_grid,
    n_iter=100,
    scoring=make_scorer(recall_score, greater_is_better=True, pos_label=1),
    n_jobs=-1,
    cv=5,
    random_state=42,
    verbose=1,
    pre_dispatch=2,
)

rf_random_search.fit(X_train, y_train)

generate_cm(y_val, rf_random_search.predict(X_val))
generate_cm(y_test, rf_random_search.predict(X_test))

# Save the best parameters for Logistic Regression
with open('rf_best_params.json', 'w') as f:
    json.dump(rf_random_search.best_params_, f)

# Clear memory
del rf_random_search
gc.collect()


In [ ]:
# Create a Logistic Regression classifier
logreg_clf = LogisticRegression(random_state=42, max_iter=1000)

# Define the parameter space for Logistic Regression
logreg_param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'None'],
    'C': list(np.logspace(-4, 4, 1000)),
    'fit_intercept': [True, False],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'l1_ratio': list(np.linspace(0, 1, 1000)),
}

# Set up randomized search for Logistic Regression
logreg_random_search = RandomizedSearchCV(
    estimator=logreg_clf,
    param_distributions=logreg_param_grid,
    n_iter=100,
    scoring=make_scorer(recall_score, greater_is_better=True, pos_label=1),
    n_jobs=-1,
    cv=5,
    random_state=42,
    verbose=1,
    pre_dispatch=2,
)

# Run the randomized search
logreg_random_search.fit(X_train, y_train)

generate_cm(y_val, logreg_random_search.predict(X_val))
generate_cm(y_test, logreg_random_search.predict(X_test))

# Save the best parameters for Logistic Regression
with open('logreg_best_params.json', 'w') as f:
    json.dump(logreg_random_search.best_params_, f)

# Clear memory
del logreg_random_search
gc.collect()


In [ ]:
# Create an SVM classifier
svm_clf = SVC(random_state=42, probability=True)

# Define the parameter space for SVM
svm_param_grid = {
    'C': list(np.logspace(-4, 4, 1000)),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': list(range(1, 6)),
    'gamma': ['scale', 'auto'] + list(np.logspace(-4, 1, 1000)),
    'coef0': list(np.linspace(0, 1, 1000)),
    'shrinking': [True, False],
}

# Set up randomized search for SVM
svm_random_search = RandomizedSearchCV(
    estimator=svm_clf,
    param_distributions=svm_param_grid,
    n_iter=100,
    scoring=make_scorer(recall_score, greater_is_better=True, pos_label=1),
    n_jobs=-1,
    cv=5,
    random_state=42,
    verbose=1,
    pre_dispatch=2,
)

# Run the randomized search
svm_random_search.fit(X_train, y_train)

# Save the best parameters for SVM
with open('svm_best_params.json', 'w') as f:
    json.dump(svm_random_search.best_params_, f)

generate_cm(y_val, svm_random_search.predict(X_val))
generate_cm(y_test, svm_random_search.predict(X_test))

# Clear memory
del svm_random_search
gc.collect()


In [ ]:
print("X_train shape: ", X_train.shape)
print("X_val shape: ", X_val.shape)
print("X_test shape: ", X_test.shape)

In [ ]:
def objective_lgbm(trial, X, y):
    
    lgbm_params = {
      'objective': 'binary',
      'boosting_type': 'gbdt',
      "metric": "binary_logloss",
      'max_depth': trial.suggest_int('max_depth', 5, 20),
      'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3),
      'num_leaves': trial.suggest_int('num_leaves', 20, 80),
      'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
      'min_child_weight': trial.suggest_float('min_child_weight', 1e-5, 100),
      'subsample': trial.suggest_float('subsample', 0.1, 1.0, step=0.1),
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
      'reg_alpha': trial.suggest_float('reg_alpha', 0, 10, step=0.1),
      'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
      'min_split_gain': trial.suggest_float('min_split_gain', 0, 1),
      'max_bin': trial.suggest_int('max_bin', 300, 800),
      'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
      'verbosity': -1,
      'seed': 42
    }


    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    num_boost_round = trial.suggest_int('num_boost_round', 50, 1000)
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        lgbm_train = lgbm.Dataset(X_train, y_train, categorical_feature=cat_cols)
        lgbm_valid = lgbm.Dataset(X_test, y_test, categorical_feature=cat_cols)#, reference=lgbm_train)
        
        #model = lgbm.LGBMClassifier(**lgbm_params)
        model = lgbm.train(lgbm_params, 
                           lgbm_train,
                           num_boost_round = num_boost_round,
                           valid_sets=[lgbm_valid],
                           callbacks=[early_stopping(stopping_rounds=100, first_metric_only=False),
                                      log_evaluation(period=10)],
        )
        preds = model.predict(X_test)
        cv_scores[idx] = log_loss(y_test, preds)
    print(f'Trial {trial.number}: Parameters: {lgbm_params}, Logloss: {np.mean(cv_scores)}')
    return -np.mean(cv_scores)
    
starting_lgbm_hp = {
  
}
#optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
#optuna.logging.set_verbosity(10)
study = optuna.create_study(direction='maximize', 
                            sampler=optuna.samplers.NSGAIISampler(seed=42), 
                            pruner=optuna.pruners.HyperbandPruner(),
                            )

warnings.filterwarnings("ignore", category=UserWarning)
study_logger = logging.getLogger("optuna")
study_logger.setLevel(logging.WARNING)
study_logger.addHandler(logging.StreamHandler())

study.enqueue_trial(starting_lgbm_hp)
study.optimize(lambda trial: objective_lgbm(trial, X_train, y_train), 
               n_trials=18000, 
               show_progress_bar=True)

best_params = study.best_params
print(f"Best log loss: {study.best_value:.4f}")
print(f"Best params: {best_params}")

lgbm_train = lgbm.Dataset(X_train, y_train, categorical_feature=cat_cols)
lgbm_val = lgbm.Dataset(X_val, y_val, categorical_feature=cat_cols)#, reference=lgbm_train)
lgbm_test = lgbm.Dataset(X_test, categorical_feature=cat_cols)#, reference=lgbm_val)

lgbm_clf = lgbm.train(best_params, lgbm_train)
y_pred = lgbm_clf.predict(X_test)

y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred]
print("y_pred_binary_len: ", len(y_pred_binary))
print("y_test shape: ", y_test.shape)
accuracy = sum(y_pred_binary == y_test.squeeze()) / len(y_test)
print(f"Accuracy on test set: {accuracy:.4f}")


lgbm_best_trial = study.best_trial
lgbm_best_params = lgbm_best_trial.params

print(f"Best score: {lgbm_best_trial.value:.4f}")
print(f"Best params: {lgbm_best_params}")

with open('trial_lgbm.pickle', 'wb') as f:
    pickle.dump(lgbm_best_params, f)


generate_cm1(y_val, lgbm_clf.predict(X_val), "XGBoost, optuna - cv, Validation Set")
generate_cm1(y_test, lgbm_clf.predict(X_test), "XGBoost, optuna - cv, Validation Set")


print("Counter - valid" , y_val.value_counts())
print("Counter - test", y_test.value_counts())


with open('lgbm_best_params.json', 'w') as f:
    json.dump(lgbm_best_params, f)



[I 2023-04-07 16:06:17,074] A new study created in memory with name: no-name-9a8a6176-f159-4c03-a2a4-45dce4a016be
/usr/local/lib/python3.9/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/18000 [00:00<?, ?it/s]

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
[110]	valid_0's binary_logloss: 0.264049
[120]	valid_0's binary_logloss: 0.259338
[130]	valid_0's binary_logloss: 0.253343
[140]	valid_0's binary_logloss: 0.248698
[150]	valid_0's binary_logloss: 0.247633
[160]	valid_0's binary_logloss: 0.247475
[170]	valid_0's binary_logloss: 0.246525
[180]	valid_0's binary_logloss: 0.245033
[190]	valid_0's binary_logloss: 0.242999
[200]	valid_0's binary_logloss: 0.243513
[210]	valid_0's binary_logloss: 0.244062
[220]	valid_0's binary_logloss: 0.24481
[230]	valid_0's binary_logloss: 0.242131
[240]	valid_0's binary_logloss: 0.241131
[250]	valid_0's binary_logloss: 0.242754
[260]	valid_0's binary_logloss: 0.243175
[270]	valid_0's binary_logloss: 0.242931
[280]	valid_0's binary_logloss: 0.245193
[290]	valid_0's binary_logloss: 0.245966
[300]	valid_0's binary_logloss: 0.24644
[310]	valid_0's binary_logloss: 0.247728
[320]	valid_0's binary_logloss: 0.248054
[330]	valid_0's binary_logloss: 0

In [ ]:
def custom_loss_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    y_pred = (y_pred > 0.5).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    custom_metric = (tn * 4) - (fn * 10)
    return 'custom_loss', custom_metric

def custom_obj(y_true, y_pred):
    grad = (y_pred - y_true)
    hess = np.ones_like(y_true)
    fn_mask = y_true == 0  # identify false negatives
    grad[fn_mask] *= 3  # triple-penalize false negatives
    hess[fn_mask] *= 2  # increase penalty for false negatives
    tn_mask = y_true == 1  # identify true negatives
    grad[tn_mask] += 2  # give bonus for true negatives
    hess[tn_mask] *= 2  # increase bonus for true negatives
    return grad, hess

class CustomMetric:
    def __call__(self, y_true, y_pred):
        y_pred = (y_pred > 0.5).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        custom_metric = (tn * 4) - (fn * 10)
        return 'custom_metric', custom_metric
custom_metric = CustomMetric()

In [11]:
def custom_obj(y_true, y_pred):
    y_pred = y_pred.get_label()
    grad = y_pred - y_true
    hess = np.ones_like(y_true)

    y_pred_labels = (y_pred > 0.5).astype(int)
    fn_mask = (y_true == 1) & (y_pred_labels == 0)  # identify false negatives
    grad[fn_mask] -= 3  # triple-penalize false negatives
    hess[fn_mask] *= 2  # increase penalty for false negatives

    tn_mask = (y_true == 0) & (y_pred_labels == 0)  # identify true negatives
    grad[tn_mask] += 2  # give bonus for true negatives
    hess[tn_mask] *= 2  # increase bonus for true negatives

    return grad, hess
class CustomMetric:
    def __call__(self, y_true, y_pred):
        y_pred = y_pred.get_label()
        y_pred = (y_pred > 0.5).astype(int)
        y_true = (y_true > 0.5).astype(int)

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        custom_metric = (tn * 4) - (fp * 4)
        return 'custom_metric', custom_metric
custom_metric = CustomMetric()

class CustomMetric2:
  def custom_eval(preds, dmatrix):
      labels = dmatrix.get_label()
      preds_binary = (preds > 0.5).astype(int)
      mcc = matthews_corrcoef(labels, preds_binary)
      fn_mask = labels == 1  # identify false negatives
      tn_mask = labels == 0  # identify true negatives
      mcc_fn = mcc * (1 - sum(fn_mask) / len(labels))  # multiply MCC by the fraction of true negatives
      mcc_tn = mcc * sum(tn_mask) / len(labels)  # multiply MCC by the fraction of false negatives
      return "mcc", (mcc_fn + mcc_tn) / 2, True
custom_metric2 = CustomMetric2()

def objective_xgb(trial,study, X, y):
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    #dtrain = xgb.DMatrix(X_train, label=y_train)
    #dtest = xgb.DMatrix(X_test, label=y_test)
    #dtrain = xgb.DMatrix(X, label=y)
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.1, 0.6),
        "tree_method": "hist",
        'objective': 'binary:logistic',
        "eval_metric": "logloss",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 0.4),
        "alpha": trial.suggest_float("alpha", 1e-8, 0.6),
        "reg_alpha": trial.suggest_float("alpha", 1e-8, 0.6),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 0.4),
        "verbosity": 0
    }

    if xgb_params["booster"] == "gbtree" or xgb_params["booster"] == "dart":
        xgb_params["max_depth"] = trial.suggest_int("max_depth", 1, 16)
        xgb_params["eta"] = trial.suggest_float("eta", 1e-8, 0.6)
        xgb_params["gamma"] = trial.suggest_float("gamma", 1e-8, 0.6)
        xgb_params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if xgb_params["booster"] == "dart":
        xgb_params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        xgb_params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        xgb_params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 0.6)
        xgb_params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 0.6)

    #pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    #pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'logloss')
    #bst = xgb.train(xgb_params, dtrain, evals=[(dtest, "validation")], callbacks=[pruning_callback])
    dtrain = xgb.DMatrix(X, label=y, enable_categorical = True)
    dval = xgb.DMatrix(X_val, label=y_val, enable_categorical = True)
    evals = [(dtrain, 'train'), (dval, 'val')]
    num_boost_round = trial.suggest_int('num_boost_round', 50, 1000)
    cv_result = xgb.cv(xgb_params, 
                       dtrain, nfold=5, 
                       num_boost_round = num_boost_round, 
                       stratified=True, 
                       early_stopping_rounds=50, 
                       seed = 42, 
                       maximize = False)
    
    logloss_train = cv_result["train-logloss-mean"].iloc[-1]
    logloss_val = cv_result["test-logloss-mean"].iloc[-1]

    min_logloss_val = cv_result["test-logloss-mean"].min()
    
    return -min_logloss_val
   
    #logloss = cv_result["test-logloss-mean"].min()
    #return logloss
    """
    mean_auc = cv_result['test-auc-mean'].max()
    #mean_error = cv_result["test-error-mean"].min()
    #return 1.0 - mean_error
    #y_preds = bst.predict(dtest)
    #y_preds_labels = np.rint(y_preds)
    #accuracy = accuracy_score(y_test, y_preds_labels)
    #return accuracy
    #return mean_auc
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    num_boost_round = trial.suggest_int('num_boost_round', 50, 500)
    # Perform cross-validation with the custom objective and evaluation metric
    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
        dval = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

        # Create a dictionary to store the evaluation results
        evals_result = {}

        # Train the model using the custom objective and evaluation metric
        bst = xgb.train(
            xgb_params,
            dtrain,
            num_boost_round=num_boost_round,
            evals=[(dtrain, 'train'), (dval, 'val')],
            obj="binary:logistic"
            custom_metric="accuracy"
            evals_result=evals_result,
            early_stopping_rounds=10,
            maximize=True,
            verbose_eval=False
        )

        # Get the best validation score
        best_val_score = max(evals_result['val']['custom_metric'])
        scores.append(best_val_score)

    # Calculate the mean validation score
    mean_val_score = np.mean(scores)

    return mean_val_score"""

starting_xgb_hp = {
    'learning_rate': 0.07937792496092369, 
    'min_child_weight': 1, 
    'subsample': 0.5072873601156911, 
    'booster': 'gbtree', 
    'lambda': 0.2101881276876542, 
    'alpha': 0.1022775152443601, 
    'reg_lambda': 0.15380633163734847, 
    'max_depth': 6, 
    'eta': 0.25908426274659535, 
    'gamma': 0.46906789773943747, 
    'grow_policy': 'lossguide', 
    'num_boost_round': 368
}
#optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
#optuna.logging.set_verbosity(10)
study = optuna.create_study(direction='maximize', 
                            sampler=optuna.samplers.NSGAIISampler(seed=42), 
                            pruner=optuna.pruners.HyperbandPruner(min_resource=1, 
                                                                  max_resource="auto", 
                                                                  reduction_factor=3),
                            )
study_logger = logging.getLogger("optuna")
study_logger.setLevel(1)
study_logger.addHandler(logging.StreamHandler())

study.enqueue_trial(starting_xgb_hp)
study.optimize(lambda trial: objective_xgb(trial, study, X_train, y_train), 
               n_trials=600, 
               show_progress_bar=True)


xgb_best_trial = study.best_trial
xgb_best_params = xgb_best_trial.params

print(f"Best score: {xgb_best_trial.value:.4f}")
print(f"Best params: {xgb_best_params}")

with open('trial_xgb.pickle', 'wb') as f:
    pickle.dump(xgb_best_params, f)


#xgb_clf = XGBClassifier(**xgb_best_params, tree_method="gpu_hist", enable_categorical=True, max_to_one_hot=1)
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical = True)
dval = xgb.DMatrix(X_val, label=y_val, enable_categorical = True)
evals = [(dtrain, 'train'), (dval, 'val')]
xgb_clf = xgb.train(xgb_best_params, dtrain, num_boost_round=1000, early_stopping_rounds=20, evals=evals)

dX_val =  xgb.DMatrix(X_val, enable_categorical = True)
dX_test = xgb.DMatrix(X_test, enable_categorical = True)
generate_cm1(y_val, xgb_clf.predict(dX_val), "XGBoost, optuna - cv, Validation Set")
generate_cm1(y_test, xgb_clf.predict(dX_test), "XGBoost, optuna - cv, Validation Set")

print("Counter - valid" , y_val.value_counts())
print("Counter - test", y_test.value_counts())



A new study created in memory with name: no-name-ec0bd533-7d40-4c09-938b-cd7718b0d99d
[I 2023-04-07 09:58:27,546] A new study created in memory with name: no-name-ec0bd533-7d40-4c09-938b-cd7718b0d99d
/usr/local/lib/python3.9/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/600 [00:00<?, ?it/s]

Trial 0 popped from the trial queue.
Trial 0 popped from the trial queue.
Trial 0 finished with value: -0.20923409850368824 and parameters: {'learning_rate': 0.07937792496092369, 'min_child_weight': 1, 'subsample': 0.5072873601156911, 'booster': 'gbtree', 'lambda': 0.2101881276876542, 'alpha': 0.1022775152443601, 'reg_lambda': 0.15380633163734847, 'max_depth': 6, 'eta': 0.25908426274659535, 'gamma': 0.46906789773943747, 'grow_policy': 'lossguide'}. Best is trial 0 with value: -0.20923409850368824.
Trial 0 finished with value: -0.20923409850368824 and parameters: {'learning_rate': 0.07937792496092369, 'min_child_weight': 1, 'subsample': 0.5072873601156911, 'booster': 'gbtree', 'lambda': 0.2101881276876542, 'alpha': 0.1022775152443601, 'reg_lambda': 0.15380633163734847, 'max_depth': 6, 'eta': 0.25908426274659535, 'gamma': 0.46906789773943747, 'grow_policy': 'lossguide'}. Best is trial 0 with value: -0.20923409850368824.


[I 2023-04-07 09:58:30,094] Trial 0 finished with value: -0.20923409850368824 and parameters: {'learning_rate': 0.07937792496092369, 'min_child_weight': 1, 'subsample': 0.5072873601156911, 'booster': 'gbtree', 'lambda': 0.2101881276876542, 'alpha': 0.1022775152443601, 'reg_lambda': 0.15380633163734847, 'max_depth': 6, 'eta': 0.25908426274659535, 'gamma': 0.46906789773943747, 'grow_policy': 'lossguide'}. Best is trial 0 with value: -0.20923409850368824.


Trial 1 finished with value: -0.29313053644493287 and parameters: {'learning_rate': 0.37516557872851514, 'min_child_weight': 10, 'subsample': 0.4659969709057026, 'booster': 'gbtree', 'lambda': 0.023233454286443665, 'alpha': 0.5197056888031997, 'reg_lambda': 0.24044600868613342, 'max_depth': 12, 'eta': 0.012350706371636523, 'gamma': 0.581945911598098, 'grow_policy': 'depthwise'}. Best is trial 0 with value: -0.20923409850368824.
Trial 1 finished with value: -0.29313053644493287 and parameters: {'learning_rate': 0.37516557872851514, 'min_child_weight': 10, 'subsample': 0.4659969709057026, 'booster': 'gbtree', 'lambda': 0.023233454286443665, 'alpha': 0.5197056888031997, 'reg_lambda': 0.24044600868613342, 'max_depth': 12, 'eta': 0.012350706371636523, 'gamma': 0.581945911598098, 'grow_policy': 'depthwise'}. Best is trial 0 with value: -0.20923409850368824.


...

Trial 9 finished with value: -0.6376292652154159 and parameters: {'learning_rate': 0.42768068083763006, 'min_child_weight': 9, 'subsample': 0.5303652916281717, 'booster': 'gblinear', 'lambda': 0.088843131967214, 'alpha': 0.071919229201556, 'reg_lambda': 0.13504607518529949}. Best is trial 0 with value: -0.20923409850368824.
Trial 9 finished with value: -0.6376292652154159 and parameters: {'learning_rate': 0.42768068083763006, 'min_child_weight': 9, 'subsample': 0.5303652916281717, 'booster': 'gblinear', 'lambda': 0.088843131967214, 'alpha': 0.071919229201556, 'reg_lambda': 0.13504607518529949}. Best is trial 0 with value: -0.20923409850368824.


[I 2023-04-07 10:07:31,861] Trial 9 finished with value: -0.6376292652154159 and parameters: {'learning_rate': 0.42768068083763006, 'min_child_weight': 9, 'subsample': 0.5303652916281717, 'booster': 'gblinear', 'lambda': 0.088843131967214, 'alpha': 0.071919229201556, 'reg_lambda': 0.13504607518529949}. Best is trial 0 with value: -0.20923409850368824.


Trial 10 failed with parameters: {'learning_rate': 0.9429667942086066, 'min_child_weight': 4, 'subsample': 0.3593953108716831, 'booster': 'dart', 'lambda': 0.3849789183523716, 'alpha': 0.1510693849773955, 'reg_lambda': 0.19889940738446912, 'max_depth': 5, 'eta': 0.1709043037780756, 'gamma': 0.022132178043850204, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.08693693180578511, 'skip_drop': 0.29367166127201016} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-11-fdf444d8b9cb>", line 168, in <lambda>
    study.optimize(lambda trial: objective_xgb(trial, study, X_train, y_train),
  File "<ipython-input-11-fdf444d8b9cb>", line 77, in objective_xgb
    cv_result = xgb.cv(xgb_params,
  File "/usr/local/lib/python3.9/dist-packages/xgboost/training.py", 

[W 2023-04-07 10:08:27,061] Trial 10 failed with parameters: {'learning_rate': 0.9429667942086066, 'min_child_weight': 4, 'subsample': 0.3593953108716831, 'booster': 'dart', 'lambda': 0.3849789183523716, 'alpha': 0.1510693849773955, 'reg_lambda': 0.19889940738446912, 'max_depth': 5, 'eta': 0.1709043037780756, 'gamma': 0.022132178043850204, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.08693693180578511, 'skip_drop': 0.29367166127201016} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-11-fdf444d8b9cb>", line 168, in <lambda>
    study.optimize(lambda trial: objective_xgb(trial, study, X_train, y_train),
  File "<ipython-input-11-fdf444d8b9cb>", line 77, in objective_xgb
    cv_result = xgb.cv(xgb_params,
  File "/usr/local/lib/python3.9/dist-pac

KeyboardInterrupt: ignored

In [ ]:
y_pred = xgb_clf.predict(dX_val)
y_pred = (y_pred > 0.5).astype(int)
tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
print("confusion Matrix validation")
print("tn: ", tn, "fp: ", fp, "fn: ", fn, "tp: ", tp)

confusion Matrix validation
tn:  19 fp:  0 fn:  118 tp:  0


In [ ]:

train_eval_metric = evals_result['train']['custom_metric']
test_eval_metric = evals_result['test']['custom_metric']

# Visualize the evaluation metric
plt.figure(figsize=(10, 6))
plt.plot(train_eval_metric, label='Train')
plt.plot(test_eval_metric, label='Test')
plt.xlabel('Boosting round')
plt.ylabel('Custom metric')
plt.title('Evaluation metric during training process')
plt.legend()
plt.show()

NameError: ignored

In [ ]:
from sklearn.ensemble import VotingClassifier

# Load the best parameters for each classifier
with open('lgbm_best_params.pkl', 'rb') as f:
lgbm_best_params = pickle.load(f)

with open("trial_xgb.pkl", "rb") as f:
xgb_trial = pickle.load(f)

with open('cat_best_params.pkl', 'rb') as f:
cat_best_params = pickle.load(f)

with open('logreg_best_params.pkl', 'rb') as f:
logreg_best_params = pickle.load(f)

with open('rf_best_params.pkl', 'rb') as f:
rf_best_params = pickle.load(f)

with open('svm_best_params.pkl', 'rb') as f:
svm_best_params = pickle.load(f)




In [ ]:
# Instantiate classifiers with the best parameters
lgbm_clf = LGBMClassifier(random_state=42, **lgbm_best_params)
xgb_clf = XGBClassifier(use_label_encoder=False, random_state=42, tree_method='hist', n_jobs=-1, **xgb_best_params)
cat_clf = CatBoostClassifier(task_type="CPU", random_seed=42, verbose = False, **cat_best_params)
logreg_clf = LogisticRegression(random_state=42, **logreg_best_params)
rf_clf = RandomForestClassifier(random_state=42, **rf_best_params)
svm_clf = SVC(probability=True, random_state=42, **svm_best_params)

# Create a soft-voting classifier
ensemble_clf = VotingClassifier(estimators=[
    ('lgbm', lgbm_clf),
    ('xgb', xgb_clf),
    ('cat', cat_clf),
    ('logreg', logreg_clf),
    ('rf', rf_clf),
    ('svm', svm_clf)
], voting='soft')



# Train the ensemble classifier
ensemble_clf.fit(X_train, y_train)

# Calculate the recall score for the ensemble
ensemble_recall = recall_score(y_true=y_test, y_pred=ensemble_clf.predict(X_test), pos_label=1)
print(f"Ensemble recall score: {ensemble_recall}")

generate_cm(y_val, ensemble_clf.predict(X_val))
generate_cm(y_test, ensemble_clf.predict(X_test))


In [ ]:
y_pred = ensemble_clf.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

print("Confusion Matrix:")
print(cm)


In [ ]:
y_pred = ensemble_clf.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

print("Confusion Matrix:")
print(cm)

In [ ]:
precision = precision_score(y_true=y_test, y_pred=y_pred, pos_label=1)
recall = recall_score(y_true=y_test, y_pred=y_pred, pos_label=1)
f1 = f1_score(y_true=y_test, y_pred=y_pred, pos_label=1)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


In [ ]:
y_pred_proba = ensemble_clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)

fpr, tpr, _ = roc_curve(y_true=y_test, y_score=y_pred_proba)

plt.figure()
plt.plot(fpr, tpr, label=f"Ensemble Classifier (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


Addition of feauture reduction methods

In [ ]:

import os
from contextlib import redirect_stdout

ensemble_clf.fit(X_train, y_train)

def voting_classifier_feature_importances(voting_clf):
    importances = []
    for clf in voting_clf.estimators_:
        if isinstance(clf, tuple):  # Check if the clf is a tuple (name, estimator)
            clf = clf[1]
        if hasattr(clf, 'coef_'):
            importances.append(clf.coef_.flatten())
        elif hasattr(clf, 'feature_importances_'):
            importances.append(clf.feature_importances_)
    
    if not importances:
        raise ValueError("No estimators in the ensemble have `coef_` or `feature_importances_` attribute.")
    
    return np.mean(importances, axis=0)


rfe = RFECV(estimator=ensemble_clf, step=1, cv=5, scoring='recall', n_jobs=-1, importance_getter=voting_classifier_feature_importances)

with open(os.devnull, 'w') as devnull:
    with redirect_stdout(devnull):
        rfe.fit(X_train, y_train)

# Print the optimal number of features
print("Optimal number of features:", rfe.n_features_)

In [ ]:
X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)
# Print the optimal number of features
print("Optimal number of features:", rfe.n_features_)

In [ ]:
ensemble_clf.fit(X_train_rfe, y_train)
y_pred_rfe = ensemble_clf.predict(X_test_rfe)

accuracy_rfe = accuracy_score(y_test, y_pred_rfe)
report_rfe = classification_report(y_test, y_pred_rfe)
conf_matrix_rfe = confusion_matrix(y_test, y_pred_rfe)

print(f"Accuracy (with RFE): {accuracy_rfe:.4f}")
print("Classification Report (with RFE):")
print(report_rfe)
print("Confusion Matrix (with RFE):")
print(conf_matrix_rfe)


Use stacking method to combine the results of different models
Extra trees classifier and Principal component analysis

In [ ]:
ex_clf = ExtraTreesClassifier(random_state=42)

pipeline = Pipeline([
    ('pca', PCA()),
    ('ex_clf', ex_clf),
])
param_grid = {
    'pca__n_components': np.arange(1, X_train.shape[1] + 1),
    'ex_clf__n_estimators': [100, 200, 500],
    'ex_clf__max_depth': [None, 10, 20, 30],
    'ex_clf__min_samples_split': [2, 5, 10],
    'ex_clf__min_samples_leaf': [1, 2, 4],
}

random_search = RandomizedSearchCV(pipeline, param_grid, n_iter=50, cv=5, scoring='recall', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)
ex_clf_best_params = random_search.best_params_

y_pred = random_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.bar(range(X_train.shape[1]), importances[sorted_indices])
plt.xticks(range(X_train.shape[1]), sorted_indices)
plt.xlabel("Feature index")
plt.ylabel("Importance")
plt.title("Feature importances in the best ExtraTreesClassifier model")
plt.show()


Use the other classifiers to predict the test data

In [ ]:
# Instantiate classifiers

lgbm_clf = LGBMClassifier(random_state=42, **lgbm_best_params)
xgb_clf = XGBClassifier(use_label_encoder=False, random_state=42, tree_method='hist', n_jobs=-1, **xgb_best_params)
cat_clf = CatBoostClassifier(task_type="CPU", random_seed=42, **cat_best_params)
logreg_clf = LogisticRegression(random_state=42, **logreg_best_params)
rf_clf = RandomForestClassifier(random_state=42, **rf_best_params)
svm_clf = SVC(probability=True, random_state=42, **svm_best_params)

# Create pipelines with PCA and classifiers
pipelines = {
    
    'LightGBM': Pipeline([('pca', PCA()), ('clf', lgbm_clf)]),
    'XGBoost': Pipeline([('pca', PCA()), ('clf', xgb_clf)]),
    'CatBoost': Pipeline([('pca', PCA()), ('clf', cat_clf)]),
    'Logistic Regression': Pipeline([('pca', PCA()), ('clf', logreg_clf)]),
    'Random Forest': Pipeline([('pca', PCA()), ('clf', rf_clf)]),
    'SVM': Pipeline([('pca', PCA()), ('clf', svm_clf)]),
}



In [ ]:
cv_results = []

for name, pipeline in pipelines.items():
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall', n_jobs=-1)
    cv_results.append((name, scores.mean(), scores.std()))

cv_results.sort(key=lambda x: x[1], reverse=True)

print("Classifier performance using cross-validation:")
for name, mean_score, std_dev in cv_results:
    print(f"{name}: {mean_score:.4f} (+/- {std_dev:.4f})")



Stacking classifier of PCA analysis

In [ ]:
base_classifiers = [
    
    ('LightGBM', lgbm_clf),
    ('XGBoost', xgb_clf),
    ('CatBoost', cat_clf),
    ('Logistic Regression', logreg_clf),
    ('Random Forest', rf_clf),
    ('SVM', svm_clf),
    
]

meta_classifier = XGBClassifier(random_state=42)
stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier)


In [ ]:
stacking_scores = cross_val_score(stacking_clf, X_train, y_train, cv=5, scoring='recall', n_jobs=-1)
print("Stacking Classifier performance:")
print(f"Mean recall: {stacking_scores.mean():.4f}, Standard deviation: {stacking_scores.std():.4f}")


In [ ]:
stacking_clf.fit(X_train, y_train)
y_pred = stacking_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)
generate_cm(y_val, stacking_clf.predict(X_val), "Stacking Classifier")
generate_cm(y_test, stacking_clf.predict(X_test), "Stacking Classifier")


In [ ]:
#count the values of y array
from collections import Counter
print(Counter(y_test))
